# ***ASL Classification Training Pipeline***
- Data Input
- Data preprocessing  & augmentation
- EfficientNet import
- Building custom model using EfficienNet
- Two level model training strategy
- Early stopping and saving model checkpoint

## **Imports**

In [15]:
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from efficientnet.tfkeras import EfficientNetB0 

## **Dataset paths**

In [16]:
# Dataset paths (Use raw strings to avoid backslash issues)
train_data_dir = r'dataset_asl/asl_alphabet_train/asl_alphabet_train'
test_data_dir = r'dataset_asl/asl_alphabet_test/asl_alphabet_test'

## **Image Data Generator**

In [17]:
# Image Data Generator for Data Augmentation (Color Mode is now RGB)
train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    rotation_range=30,       # Rotating images for better generalization
    brightness_range=[0.8, 1.2],  # Varying brightness slightly
    validation_split=0.2     # Reserving 20% of training data for validation
)

# Validation ImageDataGenerator (with same preprocessing)
val_datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)


In [18]:
# Loading training data with shuffling and data augmentation
train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(224, 224),  # Resizing all images to 224x224 for EfficientNet
    batch_size=32,           # Uses a batch size of 32
    class_mode='categorical', # For categorical classification (ASL classes)
    color_mode='rgb',         # RGB mode for color images
    shuffle=True,             # Shuffles data at the start of each epoch
    subset='training'         # Subset for training
)

# Load validation data with shuffling
val_generator = val_datagen.flow_from_directory(
    train_data_dir,
    target_size=(224, 224),  # Resizing all images to 224x224
    batch_size=32,           # Same batch size for validation
    class_mode='categorical', # Categorical classification
    color_mode='rgb',         # RGB mode
    shuffle=True,             # Shuffles data at the start of each epoch
    subset='validation'       # Subset for validation
)

Found 69600 images belonging to 29 classes.
Found 17400 images belonging to 29 classes.


In [19]:
# Loading test data
test_datagen = ImageDataGenerator(rescale=1./255)
test_generator = test_datagen.flow_from_directory(
    test_data_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode=None,  # No labels for test set
    shuffle=False  # No need to shuffle test data
)

Found 29 images belonging to 29 classes.


## **Model Building**

In [20]:
# Loading EfficientNet base model (pretrained on ImageNet)
base_model = EfficientNetB0(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Freezing the base model layers (for transfer learning)
base_model.trainable = False

# Adding regularization and more dropout to avoid overfitting
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dropout(0.5)(x)
x = Dense(128, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.01))(x)
x = Dropout(0.4)(x)
predictions = Dense(29, activation='softmax')(x)  # 29 classes for ASL

# Creating the final model
model = Model(inputs=base_model.input, outputs=predictions)

# Compiling the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Showing model summary
model.summary()

Model: "functional_5"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_2       │ (None, 224, 224,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ stem_conv (Conv2D)  │ (None, 112, 112,  │        864 │ input_layer_2[0]… │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ stem_bn             │ (None, 112, 112,  │        128 │ stem_conv[0][0]   │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ stem_activation     │ (None, 112, 112,  │          0 │ stem_bn[0][0]     │
│ (Activation)        │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_dwconv      │ (None, 112, 112,  │        288 │ stem_activation[… │
│ (DepthwiseConv2D)   │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_bn          │ (None, 112, 112,  │        128 │ block1a_dwconv[0… │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_activation  │ (None, 112, 112,  │          0 │ block1a_bn[0][0]  │
│ (Activation)        │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_se_squeeze  │ (None, 32)        │          0 │ block1a_activati… │
│ (GlobalAveragePool… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_se_reshape  │ (None, 1, 1, 32)  │          0 │ block1a_se_squee… │
│ (Reshape)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_se_reduce   │ (None, 1, 1, 8)   │        264 │ block1a_se_resha… │
│ (Conv2D)            │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_se_expand   │ (None, 1, 1, 32)  │        288 │ block1a_se_reduc… │
│ (Conv2D)            │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_se_excite   │ (None, 112, 112,  │          0 │ block1a_activati… │
│ (Multiply)          │ 32)               │            │ block1a_se_expan… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_project_co… │ (None, 112, 112,  │        512 │ block1a_se_excit… │
│ (Conv2D)            │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_project_bn  │ (None, 112, 112,  │         64 │ block1a_project_… │
│ (BatchNormalizatio… │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block2a_expand_conv │ (None, 112, 112,  │      1,536 │ block1a_project_… │
│ (Conv2D)            │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block2a_expand_bn   │ (None, 112, 112,  │        384 │ block2a_expand_c… │
│ (BatchNormalizatio… │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block2a_expand_act… │ (None, 112, 112,  │          0 │ block2a_expand_b

 Total params: 4,217,273 (16.09 MB)

 Trainable params: 167,709 (655.11 KB)

 Non-trainable params: 4,049,564 (15.45 MB)

## **Training with early stopping**

In [21]:
# Early stopping to prevent overfitting
early_stop = EarlyStopping(monitor='val_loss', patience=7, restore_best_weights=True)

# ModelCheckpoint to save the best model
model_checkpoint = ModelCheckpoint('best_asl_model.keras', save_best_only=True)

# Training the model with callbacks
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    validation_data=val_generator,
    validation_steps=val_generator.samples // val_generator.batch_size,
    epochs=25,
    callbacks=[early_stop, model_checkpoint]
)

Epoch 1/25
2175/2175 ━━━━━━━━━━━━━━━━━━━━ 2264s 1s/step - accuracy: 0.4661 - loss: 2.6683 - val_accuracy: 0.7320 - val_loss: 1.4403
Epoch 2/25
2175/2175 ━━━━━━━━━━━━━━━━━━━━ 1s 317us/step - accuracy: 0.0000e+00 - loss: 0.0000e+00 - val_accuracy: 0.6667 - val_loss: 1.4759
Epoch 3/25
2175/2175 ━━━━━━━━━━━━━━━━━━━━ 1817s 834ms/step - accuracy: 0.6391 - loss: 1.6797 - val_accuracy: 0.7743 - val_loss: 1.3409
Epoch 4/25
2175/2175 ━━━━━━━━━━━━━━━━━━━━ 1s 476us/step - accuracy: 0.0000e+00 - loss: 0.0000e+00 - val_accuracy: 0.8333 - val_loss: 1.2407
Epoch 5/25
2175/2175 ━━━━━━━━━━━━━━━━━━━━ 1686s 774ms/step - accuracy: 0.6473 - loss: 1.6464 - val_accuracy: 0.7916 - val_loss: 1.2188
Epoch 6/25
2175/2175 ━━━━━━━━━━━━━━━━━━━━ 0s 158us/step - accuracy: 0.0000e+00 - loss: 0.0000e+00 - val_accuracy: 0.6250 - val_loss: 1.4811
Epoch 7/25
2175/2175 ━━━━━━━━━━━━━━━━━━━━ 1706s 784ms/step - accuracy: 0.6504 - loss: 1.6255 - val_accuracy: 0.7895 - val_loss: 1.2210
Epoch 8/25
2175/2175 ━━━━━━━━━━━━━━━━━━━━ 1

## **Model fine-tuning by unfreezing some layers**

In [24]:
# Unfreezing some layers for fine-tuning
for layer in base_model.layers[-20:]:  # Unfreezes last 20 layers
    layer.trainable = True

# Compiling the model again with a lower learning rate
model.compile(optimizer=Adam(1e-5), loss='categorical_crossentropy', metrics=['accuracy'])

# Fine-tuning the model
history_fine_tune = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    validation_data=val_generator,
    validation_steps=val_generator.samples // val_generator.batch_size,
    epochs=17
)


Epoch 1/7
2175/2175 ━━━━━━━━━━━━━━━━━━━━ 1956s 894ms/step - accuracy: 0.9020 - loss: 0.6538 - val_accuracy: 0.9233 - val_loss: 0.5521
Epoch 2/7
2175/2175 ━━━━━━━━━━━━━━━━━━━━ 1s 303us/step - accuracy: 0.0000e+00 - loss: 0.0000e+00 - val_accuracy: 0.9167 - val_loss: 0.6552
Epoch 3/7
2175/2175 ━━━━━━━━━━━━━━━━━━━━ 1968s 904ms/step - accuracy: 0.9108 - loss: 0.5952 - val_accuracy: 0.9240 - val_loss: 0.5137
Epoch 4/7
2175/2175 ━━━━━━━━━━━━━━━━━━━━ 0s 201us/step - accuracy: 0.0000e+00 - loss: 0.0000e+00 - val_accuracy: 0.8750 - val_loss: 0.4991
Epoch 5/7
2175/2175 ━━━━━━━━━━━━━━━━━━━━ 1994s 916ms/step - accuracy: 0.9208 - loss: 0.5359 - val_accuracy: 0.9322 - val_loss: 0.4673
Epoch 6/7
2175/2175 ━━━━━━━━━━━━━━━━━━━━ 0s 201us/step - accuracy: 0.0000e+00 - loss: 0.0000e+00 - val_accuracy: 0.9583 - val_loss: 0.4496
Epoch 7/7
2175/2175 ━━━━━━━━━━━━━━━━━━━━ 2392s 1s/step - accuracy: 0.9313 - loss: 0.4826 - val_accuracy: 0.9374 - val_loss: 0.4270


## **Saving best model**

In [25]:
# Saving the fine-tuned model
model.save('efficientnet_hand_gesture_model.keras')

# Evaluating the model
test_loss, test_acc = model.evaluate(val_generator)
print(f"Test Accuracy: {test_acc * 100:.2f}%")

544/544 ━━━━━━━━━━━━━━━━━━━━ 239s 440ms/step - accuracy: 0.9374 - loss: 0.4243
Test Accuracy: 93.74%


In [26]:
model.save('efficientnet_hand_gesture_model.h5')